In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
sys.path.insert(0, '../scripts')
from utils import *

In [2]:
plt.style.use('seaborn')

In [3]:
plt.rcParams['figure.figsize'] = (12, 10)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using MXNet backend.


In [6]:
import datetime
import time

In [7]:
from keras.models import Model
from keras.layers import (Input, Bidirectional, LSTM, Merge, Flatten, Dense, Reshape, Dropout,
                          BatchNormalization, Lambda)
from keras.layers.advanced_activations import PReLU
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.engine.topology import merge

In [8]:
train_data = pd.read_csv('../data/clean-train.csv').dropna(axis=0)
test_data = pd.read_csv('../data/clean-test.csv').dropna(axis=0)

In [9]:
q1_list = train_data['q1'].tolist()
q2_list = train_data['q2'].tolist()

In [10]:
q1_list_test = test_data['q1'].tolist()
q2_list_test = test_data['q2'].tolist()

In [11]:
vocab_size = 10000

In [12]:
token = Tokenizer(nb_words=vocab_size)
token.fit_on_texts(q1_list + q2_list)

In [13]:
question1_seq = token.texts_to_sequences(q1_list)
question2_seq = token.texts_to_sequences(q2_list)

In [14]:
question1_seq_test = token.texts_to_sequences(q1_list_test)
question2_seq_test = token.texts_to_sequences(q2_list_test)

In [15]:
MAX_SEQUENCE_LENGTH = 25

In [16]:
q1_data = pad_sequences(question1_seq, maxlen=MAX_SEQUENCE_LENGTH)
q2_data = pad_sequences(question2_seq, maxlen=MAX_SEQUENCE_LENGTH)

In [17]:
q1_data_test = pad_sequences(question1_seq_test, maxlen=MAX_SEQUENCE_LENGTH)
q2_data_test = pad_sequences(question2_seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [18]:
train_labels = train_data['is_same'].values

In [19]:
test_labels = test_data['is_same'].values

In [20]:
MAX_SEQUENCE_LENGTH = 25
WORD_EMBEDDING_DIM = 300
SENT_EMBEDDING_DIM = 128
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.1
RNG_SEED = 13371447
NB_EPOCHS = 150
DROPOUT = 0.2
BATCH_SIZE = 516
nb_words = vocab_size

In [21]:
gpus = 8
gpu_list = []
for i in range(gpus):
    gpu_list.append('gpu({:.0f})'.format(i))

In [55]:
question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

q1 = Embedding(nb_words + 1, 
                 WORD_EMBEDDING_DIM, 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=True)(question1)
q1 = Bidirectional(LSTM(SENT_EMBEDDING_DIM, return_sequences=True), merge_mode="sum")(q1)

q2 = Embedding(nb_words + 1, 
                 WORD_EMBEDDING_DIM, 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=True)(question2)
q2 = Bidirectional(LSTM(SENT_EMBEDDING_DIM, return_sequences=True), merge_mode="sum")(q2)

angle = merge([q1,q2], mode='dot', dot_axes=1)
angle = Flatten()(angle)
angle = Dense((MAX_SEQUENCE_LENGTH*SENT_EMBEDDING_DIM))(angle)
angle = Reshape((MAX_SEQUENCE_LENGTH, SENT_EMBEDDING_DIM))(angle)

minus_q2 = Lambda(lambda x: -x)(q2)
distance = merge([q1,minus_q2], mode='sum', dot_axes=1)
distance = Lambda(lambda x: K.abs(x))(distance)
distance = Flatten()(distance)
distance = Dense((MAX_SEQUENCE_LENGTH*SENT_EMBEDDING_DIM))(distance)
distance = Reshape((MAX_SEQUENCE_LENGTH, SENT_EMBEDDING_DIM))(distance)

merged = merge([q1,angle, distance], mode='concat', concat_axis=1)
merged = Flatten()(merged)
merged = Dense(200)(merged)
merged = PReLU()(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200)(merged)
merged = PReLU()(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200)(merged)
merged = PReLU()(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200)(merged)
merged = PReLU()(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

is_duplicate = Dense(1, activation='sigmoid')(merged)

/home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/Keras-1.2.2-py3.6.egg/keras/layers/core.py:638: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  arg_spec = inspect.getargspec(self.function)


In [56]:
model = Model([question1,question2], is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'], context=gpu_list)

In [57]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_acc', patience=1)
mc = ModelCheckpoint(filepath='../Saved Models/lstm-angle-distance', save_best_only=True, verbose=1)

In [58]:
history = model.fit([q1_data, q2_data], train_labels, nb_epoch=100, validation_split=0.2, callbacks=[es, mc])

Train on 258726 samples, validate on 64682 samples
Epoch 1/100


RuntimeError: simple_bind error. Arguments:
input_7: (4, 25)
input_8: (4, 25)
[06:40:08] src/storage/./pooled_storage_manager.h:102: cudaMalloc failed: out of memory

Stack trace returned 10 entries:
[bt] (0) /home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x1d57cc) [0x7fa96dc157cc]
[bt] (1) /home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x1242238) [0x7fa96ec82238]
[bt] (2) /home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x1244c0a) [0x7fa96ec84c0a]
[bt] (3) /home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xe5536b) [0x7fa96e89536b]
[bt] (4) /home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xe5613d) [0x7fa96e89613d]
[bt] (5) /home/ubuntu/src/anaconda3/envs/keras1.2_p3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xe5d674) [0x7fa96e89d674]
[bt] (6) /home/ubuntu/src/anaconda3/

In [ ]:
#plt.plot(np.arange(1, 151), model.history.history['acc'], label='Training')
plt.plot(np.arange(1, 151), model.history.history['val_acc'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy History')
plt.legend()

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Log Loss')
plt.title('Loss History')
plt.legend()

In [ ]:
q1_list_test = test_data['q1'].tolist()
q2_list_test = test_data['q2'].tolist()

question1_seq_test = token.texts_to_sequences(q1_list_test)
question2_seq_test = token.texts_to_sequences(q2_list_test)

q1_data_test = pad_sequences(question1_seq_test, maxlen=MAX_SEQUENCE_LENGTH)
q2_data_test = pad_sequences(question2_seq_test, maxlen=MAX_SEQUENCE_LENGTH)

test_labels = test_data['is_same'].values

In [ ]:
preds = model.predict([q1_data_test, q2_data_test]).flatten()

In [ ]:
classes = (preds > 0.5).astype('int').flatten()

In [ ]:
generate_report(test_labels, classes.flatten(), preds.flatten())